In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader

import numpy as np

In [9]:
training_generator = DataLoader(128, "train")
val_generator = DataLoader(128, "val")

In [10]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=2)

In [13]:
halfAcc = Dense(256, clipped_relu)
i1 = tf.keras.Input((64*64*12,))
i2 = tf.keras.Input((64*64*12,))
acc1 = halfAcc(i1)
acc2 = halfAcc(i2)
layerStack = tf.keras.layers.Concatenate()([acc1, acc2])

out = Dense(384, 'sigmoid')

layerStack = out(layerStack)

model = tf.keras.Model(inputs=[i1, i2], outputs=layerStack)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['mae', 'accuracy'])

In [ ]:
model = tf.keras.models.load_model("production/")

In [14]:
#9223/9223 [==============================] - 974s 106ms/step - loss: 3.7754 - accuracy: 0.1734 - val_loss: 3.5380 - val_accuracy: 0.1943
#9223/9223 [==============================] - 940s 102ms/step - loss: 3.1249 - accuracy: 0.2536 - val_loss: 3.5691 - val_accuracy: 0.2021
model.fit(training_generator, validation_data=val_generator, epochs=1)

7170/7170 [==============================] - 157s 22ms/step - loss: 4.2619 - mae: 0.0844 - accuracy: 0.1089 - val_loss: 4.0179 - val_mae: 0.0438 - val_accuracy: 0.1301


In [6]:
import time
model.save("models/model-{}".format(int(time.time())))

2022-03-25 11:51:58.776110: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model-1648223518/assets


In [15]:
model.save("production/")

2022-04-04 11:27:55.888339: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: production/assets
